In [106]:
import tiktoken
import torch

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")
print(enc)

<Encoding 'cl100k_base'>


In [107]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [108]:
print("charachetr length"  ,len(text))

charachetr length 5370734


In [109]:
print(text[:1000])

name: A Channel plot summary:  Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
name: A Channel Ova  plot summary:  Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
name: A Channel Special  plot summary:  Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
name: A Day Before Us plot summary:  A Day Before Us is an animated Korean drama of a special and sweet love story of the following four main characters; Yeo Reum, Ha Eun, Kim Wook, and Yeon Woo.
name: A Disguised Princess plot summary:  "Ken always disguised herself as a man. She was totally confused about why she got entangled with that 

In [110]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz|~ ­°±²´½ÄÈÉÖ×ßàáâãäçèéëíïñóöùüĐēīōūΑΔΨΩθμЯоขงตนบยาเแ่้ấểệ​‐–—―‘’‚“”†…⁠Ⅲ→∞△★☆♡♥♪♭【】いうかじずたつてなのみゃゆよイウォカスダチテナハバミモル・ー一三也井優儿分勝勢原命咒园城好婆婿子季学宝寿山崎幸幼废座式归役意愛我戦書术来柴極武江玉百的福秋科立第絵老舞花袭襲詢講赘車逆道里闘靜養鬼麻龍생역인전﻿！－：＜＞
283


In [111]:
data = torch.tensor(enc.encode(text))

In [112]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [113]:
block_size = 8
train_data[:block_size+1]

tensor([  609,    25,   362, 13740,  7234, 12399,    25,   220,  2057])

In [114]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([609]) the target: 25
when input is tensor([609,  25]) the target: 362
when input is tensor([609,  25, 362]) the target: 13740
when input is tensor([  609,    25,   362, 13740]) the target: 7234
when input is tensor([  609,    25,   362, 13740,  7234]) the target: 12399
when input is tensor([  609,    25,   362, 13740,  7234, 12399]) the target: 25
when input is tensor([  609,    25,   362, 13740,  7234, 12399,    25]) the target: 220
when input is tensor([  609,    25,   362, 13740,  7234, 12399,    25,   220]) the target: 2057


In [115]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[12399,    25,   220,   362,  8334,  5334,  6532,   304],
        [  520,   682,  2853,   627,   609,    25,  1556,    72],
        [ 6134,   311,  1461,   627,   609,    25,   452,   347],
        [ 1712, 23164,     1,   449,   279,  2410,   311, 37735]])
targets:
torch.Size([4, 8])
tensor([[   25,   220,   362,  8334,  5334,  6532,   304,   459],
        [  682,  2853,   627,   609,    25,  1556,    72, 13080],
        [  311,  1461,   627,   609,    25,   452,   347,   373],
        [23164,     1,   449,   279,  2410,   311, 37735,   892]])
----
when input is [12399] the target: 25
when input is [12399, 25] the target: 220
when input is [12399, 25, 220] the target: 362
when input is [12399, 25, 220, 362] the target: 8334
when input is [12399, 25, 220, 362, 8334] the target: 5334
when input is [12399, 25, 220, 362, 8334, 5334] the target: 6532
when input is [12399, 25, 220, 362, 8334, 5334, 6532] the target: 304
when input is [12399, 25, 220, 362, 8

In [116]:

# import torch
# import torch.nn as nn
# from torch.nn import functional as F
# torch.manual_seed(1337)
# class BigramLanguangeModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
#     def forward(self, idx, targets):
#         logits = self.token_embedding_table(idx)
        
        
#         loss = F.cross_entropy(logits, targets)
#         return logits
    
# m = BigramLanguangeModel(vocab_size)
# logits = m(xb, yb)
# print(logits.shape)

In [118]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
#         B,T,C = logits.shape
#         logits= logits.view(B*T, C)
#         targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits  
    
# Define the vocab_size, idx, targets, xb, and yb
# vocab_size = 100
# idx = torch.tensor([1, 2, 3])
# targets = torch.tensor([0, 1, 0])
# xb = torch.tensor([1, 2, 3])
# yb = torch.tensor([0, 1, 0])

# Create an instance of the BigramLanguageModel
m = BigramLanguageModel(vocab_size)

# Debugging Step 1: Verify the shapes and data types of variables
print(f"idx shape: {idx.shape}, type: {idx.dtype}")
print(f"targets shape: {targets.shape}, type: {targets.dtype}")
print(f"xb shape: {xb.shape}, type: {xb.dtype}")
print(f"yb shape: {yb.shape}, type: {yb.dtype}")
print("vocab_size", vocab_size)

# Debugging Step 2: Check the size of the embedding table
print(f"Embedding table size: {m.token_embedding_table.weight.size()}")

# Debugging Step 3: Verify the compatibility of input tensors with the model architecture
logits = m(idx, targets)

# Debugging Step 4: Check the shape of the logits tensor
print(f"logits shape: {logits.shape}")
logits = m(xb, yb)
# print(logits.shape)
# print(loss)

idx shape: torch.Size([3]), type: torch.int64
targets shape: torch.Size([3]), type: torch.int64
xb shape: torch.Size([4, 8]), type: torch.int64
yb shape: torch.Size([4, 8]), type: torch.int64
vocab_size 283
Embedding table size: torch.Size([283, 283])
logits shape: torch.Size([3, 283])


IndexError: index out of range in self

torch.Size([3, 100])
